## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Shingu,JP,33.7333,135.9833,81.91,86,5,6.82,clear sky
1,1,Eureka,US,40.8021,-124.1637,59.36,94,73,4.03,broken clouds
2,2,Jiuquan,CN,39.7432,98.5174,74.55,32,93,4.76,overcast clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,33.46,69,90,24.16,light snow
4,4,Clyde River,CA,70.4692,-68.5914,37.63,100,90,11.50,light intensity drizzle


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum daily temperature you'd like during your trip?"))
max_temp = float(input("What is the maximum daily temperature you'd like during your trip?"))

What is the minimum daily temperature you'd like during your trip?80
What is the maximum daily temperature you'd like during your trip?85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
matching_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
matching_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Shingu,JP,33.7333,135.9833,81.91,86,5,6.82,clear sky
12,12,Champerico,GT,14.3000,-91.9167,82.35,86,100,4.92,overcast clouds
16,16,Mangrol,IN,21.1167,70.1167,82.42,78,65,9.37,broken clouds
24,24,Miri,MY,4.4148,114.0089,82.54,89,40,0.00,light rain
28,28,Pozo Colorado,PY,-23.4923,-58.7972,84.25,24,0,11.27,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
matching_cities_df.count()

City_ID                102
City                   102
Country                101
Lat                    102
Lng                    102
Max Temp               102
Humidity               102
Cloudiness             102
Wind Speed             102
Current Description    102
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = matching_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Shingu,JP,81.91,clear sky,33.7333,135.9833,
12,Champerico,GT,82.35,overcast clouds,14.3000,-91.9167,
16,Mangrol,IN,82.42,broken clouds,21.1167,70.1167,
24,Miri,MY,82.54,light rain,4.4148,114.0089,
28,Pozo Colorado,PY,84.25,clear sky,-23.4923,-58.7972,
29,Hithadhoo,MV,82.45,overcast clouds,-0.6000,73.0833,
33,Mahajanga,MG,84.22,few clouds,-15.7167,46.3167,
34,Touros,BR,83.30,broken clouds,-5.1989,-35.4608,
38,Kavieng,PG,82.33,moderate rain,-2.5744,150.7967,
40,Itoman,JP,84.58,broken clouds,26.1247,127.6694,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Shingu,JP,81.91,clear sky,33.7333,135.9833,Shingu UI Hotel
12,Champerico,GT,82.35,overcast clouds,14.3000,-91.9167,"Hotel y restaurante ""El Diamante"""
16,Mangrol,IN,82.42,broken clouds,21.1167,70.1167,Hotel The Royal Honours
24,Miri,MY,82.54,light rain,4.4148,114.0089,"Mega Hotel, Miri"
28,Pozo Colorado,PY,84.25,clear sky,-23.4923,-58.7972,Hospedaje la Amistad


In [23]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30,31),zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))